In [ ]:
import pandas as pd
import os
from datetime import datetime

# Function to clean and prepare data
def clean_and_prepare_data(input_folder, output_file):
    data = []
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as file:
                content = json.load(file)
                articles = content.get("articles", [])
                for article in articles:
                    pub_date = article.get("pubDate")
                    description = article.get("description", "")
                    if pub_date:
                        try:
                            dt = datetime.fromisoformat(pub_date.replace("Z", "+00:00"))
                            rounded_time = dt.replace(minute=0, second=0, microsecond=0)
                            formatted_date = rounded_time.strftime("%Y-%m-%d %H:%M:%S")
                        except ValueError:
                            formatted_date = None
                    data.append({"pub_date": formatted_date, "description": description})
    
    df = pd.DataFrame(data).drop_duplicates(subset=['pub_date', 'description'])
    df.replace(["", "NULL", "N/A"], pd.NA, inplace=True)
    df.dropna(inplace=True)
    # Check the number of rows after removing duplicates
    final_row_count = df.shape[0]
    print(f"Number of rows after removing duplicates: {final_row_count}")
    df.to_excel(output_file, index=False)
    print(f"Cleaned data saved to {output_file}")


def process_xauusd_data(input_file: str, output_file: str):
    """
    Processes the XAU/USD hourly data by combining DATE and TIME into a TIMESTAMP column and saves the result.
    
    Parameters:
        input_file (str): Path to the input Excel file containing XAU/USD data.
        output_file (str): Path to the output Excel file to save the processed data.
    """
    # Check if the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # Load the Excel file
    df = pd.read_excel(input_file)

    # Combine DATE and TIME columns into a new TIMESTAMP column
    df['TIMESTAMP'] = pd.to_datetime(df['DATE'].astype(str) + ' ' + df['TIME'].astype(str), format='%Y.%m.%d %H:%M:%S')

    # Drop the original DATE and TIME columns
    df.drop(['DATE', 'TIME', 'TICKVOL', 'VOL'], axis=1, inplace=True)

    # Reorder columns to place TIMESTAMP at the beginning
    df = df[['TIMESTAMP', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'SPREAD']]

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the result to a new Excel file
    df.to_excel(output_file, index=False)

    print(f"Processed data with TIMESTAMP has been saved to {output_file}")



def merge_xauusd_and_sentiment(price_file: str, sentiment_file: str, output_file: str):
    """
    Merges the XAU/USD price data with sentiment scores based on the TIMESTAMP and date columns, then saves the result.
    
    Parameters:
        price_file (str): Path to the input Excel file containing the XAU/USD price data.
        sentiment_file (str): Path to the input Excel file containing sentiment data.
        output_file (str): Path to the output Excel file to save the merged data.
    """
    # Check if the input files exist
    if not os.path.exists(price_file):
        raise FileNotFoundError(f"Price file not found: {price_file}")
    if not os.path.exists(sentiment_file):
        raise FileNotFoundError(f"Sentiment file not found: {sentiment_file}")

    # Load the two Excel files
    price_data = pd.read_excel(price_file)
    sentiment_data = pd.read_excel(sentiment_file)

    # Convert TIMESTAMP and date columns to datetime for proper merging
    price_data['TIMESTAMP'] = pd.to_datetime(price_data['TIMESTAMP'])
    sentiment_data['date'] = pd.to_datetime(sentiment_data['date'])

    # Perform the left join to merge sentiment scores into price data
    merged_data = price_data.merge(sentiment_data, how='left', left_on='TIMESTAMP', right_on='date')

    # Drop the extra 'date' column as we only need TIMESTAMP in the final result
    merged_data = merged_data.drop(columns=['date'])

    merged_data['Sentiment Score'] = merged_data['Sentiment Score'].fillna(method='ffill')
    merged_data['Sentiment Score'] = merged_data['Sentiment Score'].fillna(0)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the merged data back to a new Excel file
    merged_data.to_excel(output_file, index=False)

    print(f"Merged data saved to {output_file}")

def process_crude_oil_data(input_file: str, output_file: str, start_date: str, end_date: str):
    """
    Processes crude oil hourly data by combining date and time into a datetime column, filtering by date range, and saves the result.
    
    Parameters:
        input_file (str): Path to the input CSV file containing crude oil data.
        output_file (str): Path to the output Excel file to save the filtered data.
        start_date (str): Start date for filtering data in 'YYYY-MM-DD' format.
        end_date (str): End date for filtering data in 'YYYY-MM-DD' format.
    """
    # Check if the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # Load the CSV file, specifying the delimiter and column names
    data = pd.read_csv(input_file, delimiter=';', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])

    # Combine date and time columns into a single datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format='%d/%m/%Y %H:%M:%S')

    # Drop the original date, time, and unnecessary columns, keeping only datetime and close
    data = data[['datetime', 'close']]

    # Filter data for the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the filtered data to a new Excel file
    filtered_data.to_excel(output_file, index=False)

    print(f"Filtered crude oil data saved to {output_file}")




def process_vix_data(input_file: str, output_file: str, start_date: str, end_date: str):
    """
    Processes VIX hourly data by combining date and time into a datetime column, filtering by date range, and saves the result.
    
    Parameters:
        input_file (str): Path to the input CSV file containing VIX data.
        output_file (str): Path to the output Excel file to save the filtered data.
        start_date (str): Start date for filtering data in 'YYYY-MM-DD' format.
        end_date (str): End date for filtering data in 'YYYY-MM-DD' format.
    """
    # Check if the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # Load the CSV file, specifying the delimiter and column names
    data = pd.read_csv(input_file, delimiter=';', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])

    # Combine date and time columns into a single datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format='%d/%m/%Y %H:%M:%S')

    # Drop the original date, time, and unnecessary columns, keeping only datetime and close
    data = data[['datetime', 'close']]

    # Filter data for the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the filtered data to a new Excel file
    filtered_data.to_excel(output_file, index=False)

    print(f"Filtered VIX data saved to {output_file}")





def process_sp500_data(input_file: str, output_file: str, start_date: str, end_date: str):
    """
    Processes S&P 500 hourly data by combining date and time into a datetime column, filtering by date range, and saves the result.
    
    Parameters:
        input_file (str): Path to the input CSV file containing S&P 500 data.
        output_file (str): Path to the output Excel file to save the filtered data.
        start_date (str): Start date for filtering data in 'YYYY-MM-DD' format.
        end_date (str): End date for filtering data in 'YYYY-MM-DD' format.
    """
    # Check if the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    # Load the CSV file, specifying the delimiter and column names
    data = pd.read_csv(input_file, delimiter=';', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])

    # Combine date and time columns into a single datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format='%d/%m/%Y %H:%M:%S')

    # Drop the original date, time, and unnecessary columns, keeping only datetime and close
    data = data[['datetime', 'close']]

    # Filter data for the specified date range
    filtered_data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the filtered data to a new Excel file
    filtered_data.to_excel(output_file, index=False)

    print(f"Filtered S&P 500 data saved to {output_file}")





def merge_datasets(crude_file: str, xau_usd_file: str, vix_file: str, sp_file: str, output_file: str):
    """
    Merges multiple datasets (XAU/USD, crude oil, VIX, S&P 500) based on the 'datetime' column and saves the final merged dataset.
    
    Parameters:
        crude_file (str): Path to the input Excel file containing crude oil data.
        xau_usd_file (str): Path to the input Excel file containing XAU/USD data.
        vix_file (str): Path to the input Excel file containing VIX data.
        sp_file (str): Path to the input Excel file containing S&P 500 data.
        output_file (str): Path to the output Excel file to save the merged data.
    """
    # Load the crude oil data
    crude_data = pd.read_excel(crude_file, parse_dates=['datetime'])
    crude_data.rename(columns={'close': 'crude_close'}, inplace=True)

    # Load the XAU/USD data
    xau_usd_data = pd.read_excel(xau_usd_file)
    xau_usd_data['datetime'] = pd.to_datetime(xau_usd_data['TIMESTAMP'])

    # Merge XAU/USD data with crude oil data on 'datetime'
    merged_data = pd.merge(xau_usd_data, crude_data[['datetime', 'crude_close']], on='datetime', how='inner')
    merged_data = merged_data.drop('TIMESTAMP', axis=1)

    # Load the VIX data
    vix_data = pd.read_excel(vix_file, parse_dates=['datetime'])
    vix_data.rename(columns={'close': 'vix_close'}, inplace=True)

    # Merge the VIX data with the previous merged data on 'datetime'
    merged_data = pd.merge(merged_data, vix_data[['datetime', 'vix_close']], on='datetime', how='inner')

    # Load the S&P 500 data
    sp_data = pd.read_excel(sp_file, parse_dates=['datetime'])
    sp_data.rename(columns={'close': 'sp_close'}, inplace=True)

    # Merge the S&P 500 data with the merged data on 'datetime'
    final_merged_data = pd.merge(merged_data, sp_data[['datetime', 'sp_close']], on='datetime', how='inner')

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Save the final merged dataset to a new file
    final_merged_data.to_excel(output_file, index=False)

    # Print the first few rows of the final merged data
    print(final_merged_data.head())







